In [1]:
#Importing packages
#
# You can delete whatever package(s) you want..
#
import os
import json
import time
import requests
import math as m
import pandas as pd
import numpy as np
import koios_python as kp



# Because most of my data sets I have made can have up to 30+ columns and 20+ rows
# code below will increase pandas defaults for max rows 
# and columns that you can display in a Juptyer Notebook
pd.options.display.max_columns = 60
pd.options.display.max_rows = 1000

## NoteBook 1


In this Notebook we will be going over how to use the koios_python library in various scenarios a developer may face. The first thing we need to do is

create a class instance for koios_python and choose what network we would like to use for our queries, in this tutorial we will use mainnet but the 

library supports all the various developer networks available including preview, preprod, guild, and mainnet.

In [2]:
# create a koios server object with network set to mainnet
koios_mainnet = kp.URLs(network='mainnet')

# let's check the current tip of the mainnet
koios_mainnet.get_tip()

[{'hash': 'e2aebdd8a24959e0c567772d01d61b72dad599cd9fd6dcff66ebaa7310764a0c',
  'epoch_no': 384,
  'abs_slot': 80597962,
  'epoch_slot': 73162,
  'block_no': 8194190,
  'block_time': 1672164253}]

### Let's go over some use case examples for koios_python

**Native Assets:**

What if you wanted to find out how many Native Assets and tokens are on the cardano blockchain? (too expensive need async XD)

In [3]:
koios_mainnet.get_asset_list()

[{'policy_id': '00000002df633853f6a47465c9496721d2d5b1291b8398016c0e87ae',
  'asset_name': '6e7574636f696e',
  'fingerprint': 'asset12h3p5l3nd5y26lr22am7y7ga3vxghkhf57zkhd'},
 {'policy_id': '000000adf8fcbdf03a5c154123aff674edf287fb13532a343b617fb2',
  'asset_name': '5853534e4654',
  'fingerprint': 'asset1ke42k9ug86yr535qwkq5edlvfdvlpfuslrmfup'},
 {'policy_id': '000006126365b110b339bd39c5b5c0fc89eb146cc186ba8c90a3c9ca',
  'asset_name': '5061696e7465724e656b6f4f7263686172643031',
  'fingerprint': 'asset1xedcm62nhn8hv0644apeydjppudtuqxm0nyvxq'},
 {'policy_id': '000006126365b110b339bd39c5b5c0fc89eb146cc186ba8c90a3c9ca',
  'asset_name': '5061696e7465724e656b6f4f7263686172643032',
  'fingerprint': 'asset1c8lfga60kvftykq9a65naml07633zqwe883gaj'},
 {'policy_id': '000006126365b110b339bd39c5b5c0fc89eb146cc186ba8c90a3c9ca',
  'asset_name': '5061696e7465724e656b6f4f7263686172643033',
  'fingerprint': 'asset1kuezrt73hk9pp93skjl2jezn3etnyhmr6ezmru'},
 {'policy_id': '000006126365b110b339bd39c5b5c0fc8

In [5]:
# my own version of the get_asset_list function
def get_asset_list(content_range="0-999"):
    url = "https://api.koios.rest/api/v0/asset_list"
    headers = {'Range': content_range}
    response = requests.get(url, headers=headers)
    try: 
        asset_list = response.json()
        pd_asset_list = pd.DataFrame(asset_list)
        return pd_asset_list
    except Exception as e:
            print(e)

In [15]:
asset_df = pd.DataFrame()
for i in range(800):
        # data = koios_mainnet.get_asset_list(content_range="{}-{}".format(i*1000,(i+1)*1000))
        # print(data)
        # print(koios_mainnet.get_asset_list(content_range="{}-{}".format(i*1000,(i+1)*1000)))
        asset_data = get_asset_list(content_range="{}-{}".format(i*1000,(i+1)*1000))
        # print(asset_data)
        if len(asset_data) > 0:
                asset_df = pd.concat([asset_df, asset_data], axis=0, join='outer')


In [16]:
asset_df

,policy_id,asset_name,fingerprint
0,00000002df633853f6a47465c9496721d2d5b1291b8398...,6e7574636f696e,asset12h3p5l3nd5y26lr22am7y7ga3vxghkhf57zkhd
1,000000adf8fcbdf03a5c154123aff674edf287fb13532a...,5853534e4654,asset1ke42k9ug86yr535qwkq5edlvfdvlpfuslrmfup
2,000006126365b110b339bd39c5b5c0fc89eb146cc186ba...,5061696e7465724e656b6f4f7263686172643031,asset1xedcm62nhn8hv0644apeydjppudtuqxm0nyvxq
3,000006126365b110b339bd39c5b5c0fc89eb146cc186ba...,5061696e7465724e656b6f4f7263686172643032,asset1c8lfga60kvftykq9a65naml07633zqwe883gaj
4,000006126365b110b339bd39c5b5c0fc89eb146cc186ba...,5061696e7465724e656b6f4f7263686172643033,asset1kuezrt73hk9pp93skjl2jezn3etnyhmr6ezmru
...,...,...,...
995,1fd6e3b849e864206e24b9fe83eef31837956337951da2...,4d656368416e696d616c73536572696573314d656d6265...,asset1lm23gun2ek0pwwyqe9t7723t9wxl9se4n34e87
996,1fd6e3b849e864206e24b9fe83eef31837956337951da2...,4d656368416e696d616c73536572696573314d656d6265...,asset1khapqm6vpxtxygjz6mmc75260cn3keyc9ku6s8
997,1fd6e3b849e864206e24b9fe83eef31837956337951da2...,4d656368416e696d616c73536572696573314d656d6265...,asset1n56d99k35s7satjn9242me87jr7fu8c5n549m6
998,1fd6e3b849e864206e24b9fe83eef31837956337951da2...,4d656368416e696d616c73536572696573314d656d6265...,asset1nsehwa793pelk6sf04h6awl00k0szywjfyy4vy


In [17]:
asset_df.drop_duplicates(subset=['fingerprint'], inplace=True)

In [18]:
# want the index to be just the number of rows in the dataframe
asset_df.index = range(len(asset_df))
asset_df

,policy_id,asset_name,fingerprint
0,00000002df633853f6a47465c9496721d2d5b1291b8398...,6e7574636f696e,asset12h3p5l3nd5y26lr22am7y7ga3vxghkhf57zkhd
1,000000adf8fcbdf03a5c154123aff674edf287fb13532a...,5853534e4654,asset1ke42k9ug86yr535qwkq5edlvfdvlpfuslrmfup
2,000006126365b110b339bd39c5b5c0fc89eb146cc186ba...,5061696e7465724e656b6f4f7263686172643031,asset1xedcm62nhn8hv0644apeydjppudtuqxm0nyvxq
3,000006126365b110b339bd39c5b5c0fc89eb146cc186ba...,5061696e7465724e656b6f4f7263686172643032,asset1c8lfga60kvftykq9a65naml07633zqwe883gaj
4,000006126365b110b339bd39c5b5c0fc89eb146cc186ba...,5061696e7465724e656b6f4f7263686172643033,asset1kuezrt73hk9pp93skjl2jezn3etnyhmr6ezmru
...,...,...,...
799995,1fd6e3b849e864206e24b9fe83eef31837956337951da2...,4d656368416e696d616c73536572696573314d656d6265...,asset1lm23gun2ek0pwwyqe9t7723t9wxl9se4n34e87
799996,1fd6e3b849e864206e24b9fe83eef31837956337951da2...,4d656368416e696d616c73536572696573314d656d6265...,asset1khapqm6vpxtxygjz6mmc75260cn3keyc9ku6s8
799997,1fd6e3b849e864206e24b9fe83eef31837956337951da2...,4d656368416e696d616c73536572696573314d656d6265...,asset1n56d99k35s7satjn9242me87jr7fu8c5n549m6
799998,1fd6e3b849e864206e24b9fe83eef31837956337951da2...,4d656368416e696d616c73536572696573314d656d6265...,asset1nsehwa793pelk6sf04h6awl00k0szywjfyy4vy


How many wallets hold a specific asset or token?

In [11]:
asset_df[0][93]

{'policy_id': '000077e4f562322fc25f79335f7b309a89347626e4d7a64598b0b927',
 'asset_name': '53756d6d65725363656e654461794f6e65303233',
 'fingerprint': 'asset18mlzwmdw2ggyj80ecg7thnjnucppuy70y7ddav'}

In [12]:
asset_df[0][95]

{'policy_id': '000077e4f562322fc25f79335f7b309a89347626e4d7a64598b0b927',
 'asset_name': '53756d6d65725363656e654461794f6e65303235',
 'fingerprint': 'asset1m3ke2eqguyqfud3gk4pu3l0m7e8078m43evewj'}

Pool information

Pool's history?

How many pools are there?

In [15]:
asset_df[0][100]

{'policy_id': '000077e4f562322fc25f79335f7b309a89347626e4d7a64598b0b927',
 'asset_name': '53756d6d65725363656e654461794f6e65303330',
 'fingerprint': 'asset1tmusxscqcpycx8ms5ywwhd9vclfqzxykgpzxml'}